In [14]:
import cv2 
import os 
from glob import glob
import json
import csv
import io
import pickle
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd
import os.path as osp

from datetime import datetime
from pathlib import Path
from functools import partial
from p_tqdm import p_map
from tqdm import tqdm 
from module.utils import imread, imfindcircles, findProjectiveTransform, str2array, circle_detection_multi_thread, find_valid_dest_circles
from module.disp_measure import displacement_measure, homography_transformation
from module.apca import run_apca_cas 

# Import circle detection and APCA parameters 
with open('params.json') as param_config_json : 
    param_config = json.load(param_config_json)



In [2]:
import json
with open('sensor_data.json') as f:
    sensors_dict = json.load(f)

In [3]:
with open('data_tmp.pkl', "rb") as f:
    sns_dict = pickle.load(f)

In [4]:
data_fr = pd.DataFrame(sns_dict)

In [71]:
writer = pd.ExcelWriter('2021.03.24.xlsx', engine='xlsxwriter')


In [72]:
sensor_info = pd.DataFrame(columns=['snsr_num','type','pier','slab','block',
                                          'sns_dir', 'sns_loc', 'disp_dir', 'install_year'])

for k, v in  data_fr.items() : 

    
    sensor_info = sensor_info.append({'snsr_num' : str(k) , 
                                      'type' : v['type'],
                                      'pier':v['pier'],
                                      'slab':v['slab'],
                                      'block':v['block'],
                                      'sns_dir':v['sns_dir'],
                                      'sns_loc':v['sns_loc'],
                                      'disp_dir':v['disp_dir'], 
                                      'install_year' : v['install_year']} , ignore_index = True)
    


sensor_info.to_excel(writer, sheet_name= 'snsr_info')


In [73]:
SNS_INFO = pd.read_csv('sensor_info.csv')

# set dataset folder directory 
data_root = '/media/sss/Seagate Backup Plus Drive/NIA_Smart_Monitoring/folderized_data'

# Sensor type 
sns_type = 'Img'

# create image sensor list from folder 
# loop along folders under Img 
# img_sns_list = []
# for folder in glob(osp.join(data_root, sns_type, '*')): 
#     img_sns_list.append(osp.basename(folder))

# img_sns_list.sort()
img_sns_list = ['002', '005', '008', '011', '014', '017', '023', '026',  
              '029', '032','035','038','041','044','047','050','062','065','068',
              '071','074','077','090','093','094','097','098','103','104','107',
              '108','111','112','115','116','119','120','123','124','127','128',
              '131','132','135','136','139','140','143','144','147','148','151',
              '152','155','156','159','160','163','164','167','168','171','172',
              '175','176','179','180','183','184','187','188','191','192','195',
              '196','199','200','203','204','207','208','211','212']
# search imgs in subdirectories 
img_folder_list = []
for sns_num in tqdm(img_sns_list) : 
    img_list = glob(osp.join(data_root, sns_type, sns_num, '*', '*', '*'))
    
    filtered_list_by_year =  [item for item in img_list if int(item.split('/')[-3]) >= 2020]
    filtered_list_by_month =  [item for item in filtered_list_by_year if int(item.split('/')[-2]) >= 9]
    filtered_list_by_date =  [item for item in filtered_list_by_month if int(item.split('/')[-1]) >= 1]
    img_folder_list.append(filtered_list_by_date)
    
    




100%|██████████| 83/83 [00:00<00:00, 228.92it/s]


In [76]:

for idx, img_folder in tqdm(enumerate(img_folder_list)): 
    disp_info = disp_info_list[idx]
    sns_num = img_folder[0].split('/')[-4]
    disp_info.to_excel(writer, sheet_name= sns_num)

83it [00:11,  7.34it/s]


In [47]:
disp_info_list = []
for img_folder in tqdm(img_folder_list): 
    
    sns_num = img_folder[0].split('/')[-4]
    disp_info = pd.DataFrame(columns=['datetime', 'circles','x_disp','y_disp'])
    
    for date_folder in img_folder: 
        # folder names 
        log_folder = osp.join(date_folder, 'logs')
        
        log_folder_exits = Path(log_folder).exists()
        
        
        if log_folder_exits : 
            
            log_folders_by_time = glob(osp.join(log_folder, '*' ))
            
            if len(log_folders_by_time) > 1 :
                log_folders_by_time.sort()
                latest_log = log_folders_by_time[-1]
                
            else: 
                latest_log = log_folders_by_time[0]
            
            
            circles_pickle_filename = osp.join(latest_log, 'circle_detection_results.pickle')
            circle_detection_folder = osp.join(latest_log, 'circle_detection_result_images')
            displacement_dict_filename = osp.join(latest_log, 'displacement_measurement.json')
            circle_detection_list = glob(osp.join(circle_detection_folder, "*.jpg"))
            
            with open(circles_pickle_filename , 'rb') as readfile:
                circles_dict = pickle.load(readfile)
                
            with open(displacement_dict_filename , 'r') as readfile:
                displacement_dict = json.load(readfile)
            
            for circle_detection_image in circle_detection_list:
                image_basename = osp.basename(circle_detection_image)
                image_basename_no_ext = image_basename.split('.')[0]
                sns_type, _ , year_month_date, hh_mm_ss = image_basename_no_ext.split('_')
#                 print(sns_type, year_month_date, hh_mm_ss)
                dest_circles = circles_dict[image_basename]
                x_disp = displacement_dict[image_basename]['x_disp']
                y_disp = displacement_dict[image_basename]['y_disp']

                disp_info = disp_info.append({'datetime' : str(year_month_date) + str(hh_mm_ss[:2]),
                                              'circles' : dest_circles,
                                              'x_disp': x_disp,
                                              'y_disp': y_disp }, ignore_index = True)
    disp_info_list.append(disp_info)    
    disp_info.to_excel(writer, sheet_name= sns_num)

100%|██████████| 83/83 [09:31<00:00,  6.88s/it]


In [10]:
for k, v in  data_fr.items() : 
    
    
    
    print(v['data'])
#     if v['type'] == 'Img' : 
#         disp_info = pd.DataFrame(columns=['datetime', 'circles','x_disp','y_disp'])
    
#         for k2, v2 in v['circles'].items():
#             for k3, v3 in v2.items():

    
#                 disp_info = disp_info.append({'datetime' : str(k2) + str(k3[:2]) , 
#                                                   'circles' : v3,
#                                                   'x_disp':v['x_disp'][k2][k3],
#                                                   'y_disp':v['y_disp'][k2][k3]} , ignore_index = True)
#         disp_info.to_excel(writer, sheet_name= k)
        
#     elif not v['temp'] != v['temp'] : 
#         disp_info = pd.DataFrame(columns=['datetime', 'temp'])
    
#         for k2, v2 in v['temp'].items():
#             for k3, v3 in v2.items():

    
#                 disp_info = disp_info.append({'datetime' : str(k2) + str(k3[:2]) , 
#                                   type                'temp' : v3} , ignore_index = True)
        
#         disp_info.to_excel(writer, sheet_name= k)
    
    
    
    

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



KeyboardInterrupt: 

In [80]:
writer.save()

In [39]:
disp_info.to_excel(writer, sheet_name= sns_num)

In [55]:


# set dataset folder directory 
data_root = '/media/sss/Seagate Backup Plus Drive/NIA_Smart_Monitoring/folderized_data'

# Sensor type 
sns_type = 'Tmp'

# create image sensor list from folder 
# loop along folders under Img 
img_sns_list = []
for folder in glob(osp.join(data_root, sns_type, '*')): 
    img_sns_list.append(osp.basename(folder))

img_sns_list.sort()
# img_sns_list = ['002', '005', '008', '011', '014', '017', '023', '026',  
#               '029', '032','035','038','041','044','047','050','062','065','068',
#               '071','074','077','090','093','094','097','098','103','104','107',
#               '108','111','112','115','116','119','120','123','124','127','128',
#               '131','132','135','136','139','140','143','144','147','148','151',
#               '152','155','156','159','160','163','164','167','168','171','172',
#               '175','176','179','180','183','184','187','188','191','192','195',
#               '196','199','200','203','204','207','208','211','212']
# search imgs in subdirectories 
img_folder_list = []
for sns_num in tqdm(img_sns_list) : 
    img_list = glob(osp.join(data_root, sns_type, sns_num, '*', '*', '*'))
    
    filtered_list_by_year =  [item for item in img_list if int(item.split('/')[-3]) >= 2020]
    filtered_list_by_month =  [item for item in filtered_list_by_year if int(item.split('/')[-2]) >= 9]
    filtered_list_by_date =  [item for item in filtered_list_by_month if int(item.split('/')[-1]) >= 1]
    img_folder_list.append(filtered_list_by_date)

100%|██████████| 57/57 [00:03<00:00, 17.23it/s]


In [79]:
for k, v in tmp_dict.items():
#     print(k)
# for idx, img_folder in tqdm(enumerate(img_folder_list)): 
    
    v.to_excel(writer, sheet_name= k)
    

In [70]:

tmp_dict = {}
for img_folder in tqdm(img_folder_list): 
    
    sns_num = img_folder[0].split('/')[-4]
    tmp_info = pd.DataFrame(columns=['datetime', 'temp'])
    
    for date_folder in img_folder: 
        
        tmp_list = glob(osp.join(date_folder, '*.tpr'))
        for tmp_file in tmp_list : 
            tmp_measured = None
            f = io.open(tmp_file, mode="r", encoding="utf-8")
            txt = f.read()
            if txt :
                tmp_measured = str2array(txt)[1]


            tmp_basename = osp.basename(tmp_file)
            sns_type = tmp_basename[:3]
            sns_id = tmp_basename[4:7]
            date = tmp_basename[8:16]
            time_ = tmp_basename[17:23]
            
            tmp_info = tmp_info.append({'datetime' : str(date) + str(time_[:2]) , 
                                          'temp' : tmp_measured} , ignore_index = True)
            
    tmp_dict[sns_num] = tmp_info


100%|██████████| 57/57 [07:30<00:00,  7.90s/it]


In [68]:
time_

'000000'